In [ ]:
# import dependencies
import pandas as pd
import requests
import time

In [ ]:
# use header for requests
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [ ]:
# define base url
base_url = "https://www.basketball-reference.com/players/"

In [ ]:
# alphabet list to help get every player name
alphabet = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z",]

In [ ]:
# create empty dataframe
df = pd.DataFrame({"Player": "", "From": "", "To": "", "Pos": "", "Ht": "", "Wt": "", "Birth Date": "", "Colleges": ""}, index = [])
df

In [ ]:
# loop through alphabet to get table for each letter/name
for letter in alphabet:
    url = base_url + letter + "/"
    r = requests.get(url, headers=header)
    print(url)
    read_df = pd.read_html(r.text)[0]
    df = pd.concat([df, read_df])
    time.sleep(1)

In [ ]:
# work on resetting index
df = df.set_index("Player")
df = df.reset_index()
df

In [ ]:
# add empty columns to df to be filled in with splits
df["First Name"] = ""
df["Last Name"] = ""

In [ ]:
# split name and test
full_name = df.loc[:, "Player"]
thingy = full_name[0].split()
thingy[1]

In [ ]:
# loop through df of names and split
first_name = []
last_name = []
first_letter_last = []
for i in range(len(full_name)):
    first_name.append(full_name[i].split(" ")[0])
    last_name.append(full_name[i].rsplit(" ")[-1])
    first_letter_last.append(full_name[i].rsplit(" ")[-1][0].lower())
first_name

In [ ]:
# fill in empty columns
df["First Name"] = first_name
f["Last Name"] = last_name
df["Last Initial"] = first_letter_last
df

## Iterrows to obtain each players average stats for every season

In [ ]:
# create empty dataframe to fill in with season average stats
df_stats = pd.DataFrame({"Player": "", "Season": "", "Age": "", "Tm": "", "Lg": "", "Pos": "", "G": "", "GS": "", "MP": "", "FG": "", "FGA": "", "FG%": "", "3P": "", "3PA": "", "3P%": "", "2P": "", "2PA": "", "2P%": "", "eFG%": "", "FT": "", "FTA": "", "FT%": "", "ORB": "", "DRB": "", "TRB": "", "AST": "", "STL": "", "BLK": "", "TOV": "", "PF": "", "PTS": "",}, index = [])
df_stats

In [ ]:
list_url_endpoints = []
counter = 1
for index, row in df.iterrows():
    # create endpoint with intials/names
    first = df.loc[index,"First Name"][0:2].lower()
    last = df.loc[index,"Last Name"][0:5].lower()
    initial = df.loc[index, "Last Initial"]
    
    # generate URL endpoint
    url = base_url + initial + "/" + last + first +"01.html"
    
    # if name in list already, then then URL
    if url in list_url_endpoints:
        url2 = base_url + initial + "/" + last + first +"02.html"
        list_url_endpoints.append(url2)
        url=url2
        if url2 in list_url_endpoints:
            url3 = base_url + initial + "/" + last + first +"02.html"
            list_url_endpoints.append(url3)
            url=url3
            
    # if not in list then use url       
    else:
        list_url_endpoints.append(url)
        url=url
    # try and except on the url   
    try:
        r = requests.get(url, headers=header)
        print(f"#{counter} of {len(df)} resuts with {r} and URL: {url}")
        read_df = pd.read_html(r.text)[0]
        read_df["Player"]=df.loc[index, 'Player']
        df_stats = pd.concat([df_stats, read_df])
        time.sleep(2)
    except:
        time.sleep(2)
    counter +=1

In [ ]:
# save base csv results
filepath = 'stats_all.csv'
df_stats.to_csv(filepath) 

In [ ]:
# dataframe to reset datatypes
df0=df_stats.apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [ ]:
# remove non numberic columns
df0=df0.drop(["Player", "Season", "Tm", "Lg", "Pos"], axis=1)
df0

In [ ]:
# create categorical df
df1=dfa[["Player", "Season", "Tm", "Lg", "Pos"]]
df1

In [ ]:
# recombine numerical and categorical df
df_0 = pd.merge(df1, df0, left_index=True, right_index=True)

In [ ]:
# remove all rows where the age is null - removing career and season summary
df_0=df_0.loc[df_0["Age"].isnull() != False]

In [ ]:
df_0.to_csv("stats_seasons.csv")

In [ ]:
# reset the index so there is an index for pandas purposes and a primary key column for SQL purposes
df_stats = df_stats.reset_index()
df_stats

In [ ]:
df_stats=df_stats.rename(columns={"index": "id"})
df_stats

In [ ]:
df_stats=pd.read_csv("stats_all.csv")

In [ ]:
# create conneciton to PostgreSQL
protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
database_name = 'nba_stats_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
# Confirm tables
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
df